### PySpark 설정

In [2]:
import findspark
findspark.init()
# [+] pyspark 불러오기
from pyspark import SparkConf, SparkContext

In [7]:
# [+] SparkConf, SparkContext 객체 생성하기
conf = SparkConf().setMaster("local").setAppName("reduction_operations")
sc = SparkContext(conf = conf)

In [9]:
sc.getConf().getAll()

[('spark.master', 'local'),
 ('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.app.name', 'reduction_operations'),
 ('spark.driver.port', '62846'),
 ('spark.executor.id', 'driver'),
 ('spark.app.id', 'local-1680575441978'),
 ('spark.app.startTime', '1680575441014'),
 ('spark

### reduce( )

In [8]:
# reduce()
sc.parallelize([1, 2, 3, 4, 5]).reduce(lambda x, y: x + y)

15

In [10]:
# reduce() 는 각 파티션별로 수행됨
# 파티션 수: 2
sc.parallelize([1, 2, 3, 4, 5], 2).reduce(lambda x, y: x + y)

15

In [14]:
# reduce() 주의점: 파티션 구성에 따라 결과 값이 달라질 수 있음
# 예: lambda x, y: (x * 2) + y
sc.parallelize([1, 2, 3, 4]).reduce(lambda x, y: (x*2) + y)

26

In [13]:
# 파티션 수: 2
sc.parallelize([1, 2, 3, 4], 2).reduce(lambda x, y: (x*2) + y)

18

### fold( )

In [15]:
# fold(): reduce()와 유사하며, 초기값을 지정
sc.parallelize([1, 2, 3, 4]).fold(1, lambda x,y: x + y)

12

In [16]:
# 파티션 수: 2
sc.parallelize([1, 2, 3, 4], 2).fold(1, lambda x,y: x + y)

13

### 10000개 정수 난수의 총합 계산하기

In [18]:
import numpy as np

In [19]:
x = np.random.randint(0, 100, size = 10000)

In [20]:
x[:100]

array([91, 63, 18, 26, 28, 95, 75, 67, 79, 24, 45, 54, 40, 14, 45, 20,  3,
       85, 28, 26, 98, 40, 26, 53, 75, 14, 49, 21, 89,  3, 46,  1, 11,  8,
       67, 52, 11, 90, 78, 83, 13, 40, 61, 17, 73, 84, 64, 60, 54, 21, 48,
       26, 26, 15, 76, 40, 43, 32, 68, 46, 50,  2, 20, 37, 43, 98, 84, 20,
       46, 25, 54, 63, 50, 89, 93, 74, 30,  2, 35, 14, 69, 99, 26, 75,  4,
       82, 22, 16, 56, 81, 15, 78, 50, 95, 54, 91, 42, 51, 75, 46])

In [21]:
rdd = sc.parallelize(x, 8)

In [25]:
partitioned_data = rdd.glom().collect()        # glom() : 파티션별로 값을 가지고 객체를 반환
for i, partition in enumerate(partitioned_data):
    print(f"Partiton {i}: {partition}\n")

Partiton 0: [91, 63, 18, 26, 28, 95, 75, 67, 79, 24, 45, 54, 40, 14, 45, 20, 3, 85, 28, 26, 98, 40, 26, 53, 75, 14, 49, 21, 89, 3, 46, 1, 11, 8, 67, 52, 11, 90, 78, 83, 13, 40, 61, 17, 73, 84, 64, 60, 54, 21, 48, 26, 26, 15, 76, 40, 43, 32, 68, 46, 50, 2, 20, 37, 43, 98, 84, 20, 46, 25, 54, 63, 50, 89, 93, 74, 30, 2, 35, 14, 69, 99, 26, 75, 4, 82, 22, 16, 56, 81, 15, 78, 50, 95, 54, 91, 42, 51, 75, 46, 21, 92, 17, 64, 14, 25, 73, 41, 41, 2, 57, 66, 21, 63, 46, 42, 36, 18, 67, 89, 82, 67, 20, 7, 41, 19, 60, 78, 54, 31, 49, 85, 22, 91, 8, 93, 58, 95, 5, 39, 57, 23, 48, 82, 27, 81, 59, 72, 81, 29, 45, 57, 77, 39, 29, 65, 0, 21, 31, 66, 0, 46, 30, 30, 44, 67, 50, 82, 55, 23, 0, 75, 86, 58, 20, 39, 71, 78, 6, 66, 81, 38, 18, 2, 4, 74, 89, 65, 19, 86, 37, 74, 0, 62, 82, 47, 95, 73, 28, 81, 3, 21, 53, 73, 40, 20, 38, 39, 26, 4, 40, 97, 48, 39, 6, 72, 18, 54, 7, 29, 83, 52, 69, 36, 59, 85, 35, 35, 53, 5, 56, 34, 46, 82, 82, 93, 81, 46, 48, 11, 97, 93, 10, 67, 10, 57, 51, 97, 54, 77, 76, 58, 38

In [26]:
rdd.reduce(lambda x,y : x + y)

489904

### grouBy()

In [30]:
# groupBy(): 인자 함수를 기준으로 그룹핑을 수행
res = sc.parallelize([1, 1, 2, 3, 5, 8]).groupBy(lambda x: x % 2).collect()

In [31]:
# groupBy() 결과 출력하기
res

[(1, <pyspark.resultiterable.ResultIterable at 0x1b545e52af0>),
 (0, <pyspark.resultiterable.ResultIterable at 0x1b545ee1130>)]

In [34]:
for k, v in res:
    print(k, list(v))

1 [1, 1, 3, 5]
0 [2, 8]


### aggregate( )

In [35]:
# reduce(), fold(): 입출력 RDD 형식이 같지 않으면 오류가 발생
sc.parallelize([1, 2, 3, 4]).reduce(lambda x, y: (x + y, x * y)) # single value -> pair 생성 시도할 경우 오류

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 15.0 failed 1 times, most recent failure: Lost task 0.0 in stage 15.0 (TID 55) (TaekwonLaptop executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Dev\Spark\spark-3.3.2-bin-hadoop3\python\lib\pyspark.zip\pyspark\worker.py", line 686, in main
  File "C:\Dev\Spark\spark-3.3.2-bin-hadoop3\python\lib\pyspark.zip\pyspark\worker.py", line 678, in process
  File "C:\Dev\Spark\spark-3.3.2-bin-hadoop3\python\lib\pyspark.zip\pyspark\serializers.py", line 273, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "C:\Dev\Spark\spark-3.3.2-bin-hadoop3\python\pyspark\rdd.py", line 1248, in func
    yield reduce(f, iterator, initial)
  File "C:\Dev\Spark\spark-3.3.2-bin-hadoop3\python\pyspark\util.py", line 81, in wrapper
    return f(*args, 

In [36]:
sc.parallelize([1, 2, 3, 4]).fold((0, 0), lambda x, y: (x[0] + y, x[1] + 1))

TypeError: unsupported operand type(s) for +: 'int' and 'tuple'

In [ ]:
"""
    aggregate(): 입출력 형식 다른 RDD 에 대한 reduction을 수행
    - zeroValues: 파티션 초기 값
    - seqOp: 각 파티션에 대한 reduction 함수
    - combop: 파티션 결과를 합치는 reduction 함수
""" 

seqOp = (lambda x, y: (x[0] + y, x[1] + 1))
combOp = (lambda x, y: (x[0] + y[0], x[1] + y[1]))

In [ ]:
# aggregate() 수행
